In [ ]:
# existing issues with data, to keep in mind...
# weather data is absent for ~12 hours at a time, often

In [ ]:
import pandas as pd
import numpy as np
import glob, geocoder, time

In [ ]:
# load, combine crime data
crimedf = pd.DataFrame(index=[],columns=[])
for counter, file in enumerate(glob.glob('raw_data/stl_crime_data/*.csv')):
    tmp = pd.read_csv(file, encoding = 'ISO-8859-1', skiprows=None, low_memory=False)
    tmp.columns=tmp.columns.str.replace('-','')
    tmp.columns=tmp.columns.str.replace(' ','')
    tmp.columns=tmp.columns.str.replace('DateOccured','Date')
    tmp.columns=tmp.columns.str.replace('DateOccur','Date')
    tmp.columns=tmp.columns.str.replace('ILEADSAddress','Address')
    tmp.columns=tmp.columns.str.replace('ILEADSStreet','Street')
    crimedf = pd.concat([crimedf,tmp], axis=0, ignore_index=True)
#crimedf = crimedf.sample(n=1000, axis=0)
crimedf = crimedf.reset_index().drop(['index'],1)

In [ ]:
# clean crime data
dropcolumnscrime = ['CADAddress','CADStreet','AdministrativeAdjustmentIndicator','Beat','CodedMonth','Complaint','DateCrimeCoded','FlagAdministrative','FlagCleanup','FlagCrime','FlagUnfounded','ILeadsAdd','ILeadsApprove','ILeadsAsg','ILeadsType','LocationComment','LocationName','MonthReportedtoMSHP','NewCrimeIndicator','UnfoundedCrimeIndicator']
dropcolumnscrime = pd.Index(dropcolumnscrime).intersection(crimedf.columns).tolist()
crimedf = crimedf.drop(dropcolumnscrime, 1)
crimedf = crimedf[crimedf['Count']==1]
crimedf = crimedf.drop('Count', 1)
crimedf['Date'] = pd.to_datetime(crimedf['Date']) # to pandas datetime format
crimedf = crimedf[crimedf['Date'].dt.year>=2008]
# most of these cases correspond to no location (on some street, but no address or neighborhood)
crimedrop = ((crimedf['Address'].isnull()) | (crimedf['Street'].str.contains('UNKNOWN')) | (crimedf['Neighborhood']==0))
crimedf = crimedf[~crimedrop]
print('Dropped ',sum(crimedrop)/crimedf.shape[0]*100,'%',' of crimes',sep='')
strcols = ['Crime','District','Description','Address','Street']
crimedf[strcols] = crimedf[strcols].astype(str)
crimedf.loc[crimedf['Crime'].str.len()==5, 'Crime'] = '0' + crimedf.loc[crimedf['Crime'].str.len()==5, 'Crime']
crimedf = crimedf.reset_index().drop(['index'],1)

In [ ]:
# find lat/lon for crimes, use google lookup tool for empty data
import stateplane
from pyproj import Proj, transform

inProj = Proj(init='esri:102696', preserve_units = True)
outProj = Proj(init='epsg:4326')

dropinds = []
for ind in crimedf.index:
    xc = crimedf['XCoord'].iloc[ind]
    yc = crimedf['YCoord'].iloc[ind]
    if (xc==0):
        addrstring = crimedf['Address'].iloc[ind] + ' ' + crimedf['Street'].iloc[ind] + ' St. Louis, MO'
        print("Looking up w/ geocoder.google", addrstring, ind/crimedf.shape[0], sep=' ')
        g = geocoder.google(addrstring)
        if not g.latlng:
            crimedf.loc[ind,'XCoord'], crimedf.loc[ind,'YCoord'] = 906515.1600962189, 1017387.5860900703
            dropinds.append(ind)
        else:
            sp = stateplane.from_latlon(g.latlng[0],g.latlng[1])
            crimedf.loc[ind, 'XCoord'], crimedf.loc[ind, 'YCoord'] = sp[0]/0.3048, sp[1]/0.3048

crimedf['XCoord'], crimedf['YCoord'] = transform(inProj, outProj, crimedf['XCoord'].tolist(), crimedf['YCoord'].tolist())

print('Dropped ',len(dropinds)/crimedf.shape[0],'% of crimes',sep='')
crimedf = crimedf.drop(dropinds)
crimedf = crimedf.reset_index().drop(['index'],1)

In [ ]:
# load and clean weather data
weatherdf = pd.read_csv('raw_data/stl_weather/stl_weather_hourly.csv', encoding = 'ISO-8859-1', skiprows=None, low_memory=False)
dropcolumnsweather = ['PeakWindDirection','HOURLYAltimeterSetting','PeakWindDirection','DAILYAverageSeaLevelPressure','STATION','STATION_NAME','ELEVATION','LATITUDE','LONGITUDE','HOURLYDRYBULBTEMPC','HOURLYWETBULBTEMPC','HOURLYDewPointTempC','HOURLYPressureTendency','HOURLYPressureChange','HOURLYSeaLevelPressure']    
weatherdf = weatherdf.drop(dropcolumnsweather, 1)
monthlydailycols = weatherdf.filter(regex='Monthly|monthly|DAILY').columns
weatherdf = weatherdf.drop(monthlydailycols, 1)
weatherdf = weatherdf[weatherdf['REPORTTPYE']!='SOD'].drop(['REPORTTPYE'],1)
weatherdf['DATE'] = pd.to_datetime(weatherdf['DATE'])
weatherdf['HOURLYDRYBULBTEMPF'] = weatherdf['HOURLYDRYBULBTEMPF'].str.replace('s','')
weatherdf['HOURLYVISIBILITY'] = weatherdf['HOURLYVISIBILITY'].str.replace('V','')
weatherdf['HOURLYDewPointTempF'] = weatherdf['HOURLYDewPointTempF'].str.replace('s','')
weatherdf['HOURLYWindSpeed'] = weatherdf['HOURLYWindSpeed'].str.replace('s','')
weatherdf['HOURLYStationPressure'] = weatherdf['HOURLYStationPressure'].str.replace('s','')
weatherdf['HOURLYPrecip'] = weatherdf['HOURLYPrecip'].str.replace('s','').str.replace('T','.01')
weatherdf['HOURLYSKYCONDITIONS'] = weatherdf['HOURLYSKYCONDITIONS'].str[0:3]
weatherdf.loc[weatherdf['HOURLYSKYCONDITIONS'].isin(['BKN','SCT','FEW','OVC','CLR'])==False,'HOURLYSKYCONDITIONS'] = 'OTH'
weatherdf = weatherdf[~weatherdf['DATE'].isnull()]
# make numeric & fill in nan values with previous readings
numcols = ['HOURLYVISIBILITY','HOURLYPRSENTWEATHERTYPE','HOURLYDRYBULBTEMPF','HOURLYWETBULBTEMPF','HOURLYDewPointTempF','HOURLYRelativeHumidity','HOURLYWindSpeed','HOURLYWindDirection','HOURLYWindGustSpeed','HOURLYStationPressure','HOURLYPrecip']
weatherdf[numcols] = weatherdf[numcols].apply(pd.to_numeric, errors='coerce', axis=1)
weatherdf = weatherdf.fillna(method='ffill')
weatherdf = weatherdf.fillna(method='bfill')
weatherdf = weatherdf.reset_index().drop(['index'],1)

In [ ]:
# make weather columns in crime dataframe
def find_nearest(array,value):
    ind = (np.abs(array-value)).argmin()
    return ind

crimedftmp = pd.DataFrame(columns=['HOURLYDRYBULBTEMPF','HOURLYWETBULBTEMPF','HOURLYVISIBILITY','HOURLYDewPointTempF','HOURLYRelativeHumidity','HOURLYWindSpeed','HOURLYWindDirection','HOURLYStationPressure','HOURLYPrecip','HOURLYSKYCONDITIONS'])
start = time.time()
for ind in crimedf.index:
    minind = find_nearest(weatherdf['DATE'], crimedf.loc[ind,'Date'])
    weatherdfminind = weatherdf.loc[minind,:]
    crimedftmp = crimedftmp.append(weatherdfminind[['HOURLYDRYBULBTEMPF','HOURLYWETBULBTEMPF','HOURLYVISIBILITY','HOURLYDewPointTempF','HOURLYRelativeHumidity','HOURLYWindSpeed','HOURLYWindDirection','HOURLYStationPressure','HOURLYPrecip','HOURLYSKYCONDITIONS']] )
    print(ind/crimedf.shape[0], sep=' ', end="\r")

end = time.time()
crimedftmp = crimedftmp.reset_index().drop('index',1)
crimedf = pd.merge(crimedf,crimedftmp, left_index=True, right_index=True)
crimedf.columns = ['Address', 'Crime', 'Date', 'Description', 'District', 'Neighborhood','Street', 'XCoord', 'YCoord', 'dry_temp','wet_temp','visibility','dew_point','humidity','wind_speed','wind_dir','pressure','precip','sky']
print('\n')
print('Took', (end-start)/3600, 'hours',sep=' ')

In [ ]:
# Load neighborhood population data
hooddict = {'Carondelet': 1, 'Patch': 2, 'Holly Hills': 3, 'Boulevard Heights': 4, 'Bevo Mill': 5, 'Princeton Heights': 6, 'South Hampton': 7, 'St. Louis Hills': 8, 'Lindenwood Park': 9, 'Ellendale': 10, 'Clifton Heights': 11, 'The Hill': 12, 'Southwest Garden': 13, 'North Hampton': 14, 'Tower Grove South': 15, 'Dutchtown': 16, 'Mount Pleasant': 17, 'Marine Villa': 18, 'Gravois Park': 19, 'Kosciusko': 20, 'Soulard': 21, 'Benton Park': 22, 'McKinley Heights': 23, 'Fox Park': 24, 'Tower Grove East': 25, 'Compton Heights': 26, 'Shaw': 27, 'Botanical Heights (formerly McRee Town)': 28, 'Tiffany': 29, 'Benton Park West': 30, 'The Gate District': 31, 'Lafayette Square': 32, 'Peabody, Darst, Webbe': 33, 'La Salle': 34, 'Downtown': 35, 'Downtown West': 36, 'Midtown': 37, 'Central West End': 38, 'Forest Park Southeast': 39, 'Kings Oak': 40, 'Cheltenham': 41, 'Clayton-Tamm': 42, 'Franz Park': 43, 'Hi-Pointe': 44, 'Wydown-Skinker': 45, 'Skinker-DeBaliviere': 46, 'DeBaliviere Place': 47, 'West End': 48, 'Visitation Park': 49, 'Wells-Goodfellow': 50, 'Academy': 51, 'Kingsway West': 52, 'Fountain Park': 53, 'Lewis Place': 54, 'Kingsway East': 55, 'The Greater Ville': 56, 'The Ville': 57, 'Vandeventer': 58, 'JeffVanderLou': 59, 'St. Louis Place': 60, 'Carr Square': 61, 'Columbus Square': 62, 'Old North St. Louis': 63, 'Near North Riverfront': 64, 'Hyde Park': 65, 'College Hill': 66, 'Fairground Neighborhood': 67, "O'Fallon": 68, 'Penrose': 69, 'Mark Twain-I-70 Industrial': 70, 'Mark Twain': 71, 'Walnut Park East': 72, 'North Point': 73, 'Baden': 74, 'Riverview': 75, 'Walnut Park West': 76, 'Covenant Blu-Grand Center': 77, 'Hamilton Heights': 78, 'North Riverfront': 79, 'Carondelet Park': 80, 'Tower Grove Park': 81, 'Forest Park': 82, 'Fairgrounds Park': 83, 'Penrose Park': 84, 'Calvary-Bellefontaine Cemetaries': 86}
inv_hooddict = {v: k for k, v in hooddict.items()}
hoodpopdf = pd.read_csv('raw_data/StL_neighborhood_populations_2010.csv', encoding = 'ISO-8859-1', skiprows=None)
hoodpopdf['Neighborhood'].replace(hooddict, inplace=True)

In [ ]:
# Save as cleaned "pickle" dataframe files
crimedf.to_pickle('cleaned_data/crimedf.pkl')
hoodpopdf.to_pickle('cleaned_data/hoodpopdf.pkl')
weatherdf.to_pickle('cleaned_data/weatherdf.pkl')